**How to select hidden layers and number of hidden neurons in ANN**

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch #Keras Tuner - Decide number of hidden layers and neuron in Neural Network

In [2]:
df = pd.read_csv(r"C:\Users\ROHIT\Desktop\Real_Combine.csv\Real_Combine.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
X = df.iloc[:,:-1]#independent features
y = df.iloc[:,-1]#dependent feature

In [4]:
X

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2
...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5


In [5]:
y

0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
1092    185.583333
Name: PM 2.5, Length: 1093, dtype: float64

# Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layer?
3. What should be the Learning Rate?

In [6]:
def build_model(hp):     #hp is the parameter when running Random Search by default it is defining the instance of model that we are creating.
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):  #neural network can have hidden layers b/w 2 to 20. 
        model.add(layers.Dense(units=hp.Int('units_' +str(i), #number of neurons ranging from min 32 to max 512 in each layer.
                                            min_value=32, #hp.Int means try with different-2 values
                                            max_value=512, 
                                            step=32), 
                                            activation='relu'))
        model.add(layers.Dense(1, activation='linear')) #output layer
        model.compile(
            optimizer = keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),  #learning rate again hyperparameter
            loss='mean_absolute_error',    #hp.Choice means out of three choose one
            metrics=['mean_absolute_error'])
        return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3, # we will try to run 15 iteration(5 x 3)
    directory='project',
    project_name='Air Quality Index')

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [10]:
tuner.search(X_train, y_train,
            epochs = 5,
            validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 10
units_0: 384
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 18
units_0: 192
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 6
units_0: 384
learning_rate: 0.01
Score: nan
Trial summary
Hyperparameters:
num_layers: 9
units_0: 32
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 18
units_0: 320
learning_rate: 0.001
Score: nan
